## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,30.20,86,40,4.61,scattered clouds
1,1,Anadyr,RU,64.7500,177.4833,59.50,38,0,7.29,clear sky
2,2,Buin,CL,-33.7333,-70.7500,64.06,41,100,2.64,overcast clouds
3,3,Upernavik,GL,72.7868,-56.1549,31.33,94,100,4.32,overcast clouds
4,4,Hobart,AU,-42.8794,147.3294,46.99,76,0,9.22,clear sky


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 72
What is the maximum temperature you would like for your trip? 80


In [23]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"]>=min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Rikitea,PF,-23.1203,-134.9692,75.42,69,66,12.44,broken clouds
9,9,Elko,US,41.0002,-115.5012,78.82,16,9,8.66,clear sky
10,10,Fez,MA,34.0372,-4.9998,77.00,47,40,5.75,scattered clouds
20,20,Elk City,US,35.4120,-99.4043,75.69,60,1,14.97,clear sky
21,21,Faanui,PF,-16.4833,-151.7500,79.92,73,100,15.86,overcast clouds


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [45]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Rikitea,PF,-23.1203,-134.9692,75.42,69,66,12.44,broken clouds
9,9,Elko,US,41.0002,-115.5012,78.82,16,9,8.66,clear sky
10,10,Fez,MA,34.0372,-4.9998,77.00,47,40,5.75,scattered clouds
20,20,Elk City,US,35.4120,-99.4043,75.69,60,1,14.97,clear sky
21,21,Faanui,PF,-16.4833,-151.7500,79.92,73,100,15.86,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
670,670,Shihezi,CN,44.3000,86.0333,77.27,24,37,6.40,scattered clouds
676,676,Aksu,CN,41.1231,80.2644,78.26,24,94,11.16,overcast clouds
678,678,Gat,IL,31.6100,34.7642,77.40,28,0,4.74,clear sky
680,680,Marzuq,YE,14.4000,46.4667,76.87,40,0,4.36,clear sky


In [65]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Rikitea,PF,75.42,broken clouds,-23.1203,-134.9692,
9,Elko,US,78.82,clear sky,41.0002,-115.5012,
10,Fez,MA,77.00,scattered clouds,34.0372,-4.9998,
20,Elk City,US,75.69,clear sky,35.4120,-99.4043,
21,Faanui,PF,79.92,overcast clouds,-16.4833,-151.7500,
40,Kununurra,AU,78.78,overcast clouds,-15.7667,128.7333,
46,Gamba,GA,75.63,few clouds,-2.6500,10.0000,
49,Mandan,US,75.18,few clouds,46.8267,-100.8896,
79,Humberto De Campos,BR,75.61,overcast clouds,-2.5983,-43.4611,
82,Atuona,PF,79.56,clear sky,-9.8000,-139.0333,


In [66]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {"radius": 5000,"type": "lodging","key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"] 
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")
hotel_df.head(10) 
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Rikitea,PF,75.42,broken clouds,-23.1203,-134.9692,Pension Maro'i
9,Elko,US,78.82,clear sky,41.0002,-115.5012,
10,Fez,MA,77.00,scattered clouds,34.0372,-4.9998,Hotel Mounia Fès
20,Elk City,US,75.69,clear sky,35.4120,-99.4043,Hampton Inn & Suites Elk City
21,Faanui,PF,79.92,overcast clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
40,Kununurra,AU,78.78,overcast clouds,-15.7667,128.7333,Hotel Kununurra
46,Gamba,GA,75.63,few clouds,-2.6500,10.0000,
49,Mandan,US,75.18,few clouds,46.8267,-100.8896,Twin City Estates
79,Humberto De Campos,BR,75.61,overcast clouds,-2.5983,-43.4611,Pousada do Cabeça
82,Atuona,PF,79.56,clear sky,-9.8000,-139.0333,Villa Enata


In [72]:
# 7. Drop the rows where there is no Hotel Name.

import numpy as np

clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] !="")]
clean_hotel_df.head()

#hotel_df.loc[(hotel_df["Hotel Name"] == "")]=np.delete()
#test=clean_hotel_df.loc[9, 'Hotel Name']
#test
#print(df.replace(r'^\s*$', np.nan, regex=True))




,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Rikitea,PF,75.42,broken clouds,-23.1203,-134.9692,Pension Maro'i
10,Fez,MA,77.00,scattered clouds,34.0372,-4.9998,Hotel Mounia Fès
20,Elk City,US,75.69,clear sky,35.4120,-99.4043,Hampton Inn & Suites Elk City
21,Faanui,PF,79.92,overcast clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
40,Kununurra,AU,78.78,overcast clouds,-15.7667,128.7333,Hotel Kununurra


In [74]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [75]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.????????why do we do this step
locations = clean_hotel_df[["Lat", "Lng"]]

In [77]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
#max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
#fig.add_layer(heat_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))